In [1]:
import pandas as pd
import numpy as np
import json
import csv
from datetime import datetime

In [2]:
menuDir = 'data-live-week5'
restsOld = pd.read_csv('%s/restaurants_prod.csv' % menuDir)
hospsOld = pd.read_csv('%s/hospitals_prod.csv' % menuDir)
seedData = pd.read_csv('%s/seed_data.csv' % menuDir)

In [3]:
restDict = {}
hospDict = {}
def restToDict(r):
    restDict[r.restName] = {"id":r.id, "userId":r.userId}
def hospToDict(r):
    hospDict[r.hospName] = {"id":r.id}

v = restsOld.apply(restToDict, axis=1)
v = hospsOld.apply(hospToDict, axis=1)

In [4]:
# Convert the date correctly
def getDate(r):
    time = ''
    if(r['meal'] == 'Lunch' and r['hospital'] == 'Pennsylvania Hospital (PAH, Pennsy)'):
        time = '12:15:00-04'
    elif(r['meal'] == 'Lunch'):
        time = '12:00:00-04'
    elif(r['meal'] == 'Dinner' and r['hospital'] == 'Pennsylvania Hospital (PAH, Pennsy)'):
        time = '17:15:00-04'
    elif(r['meal'] == 'Dinner'):
        time = '17:00:00-04'
    else:
        print(r)
    date = ["%02d" % int(v) for v in r['date'].split('/')]
    date = "%s-%s-%s" % (date[2], date[0], date[1])
    return ("%s %s" % (date, time))

seedData['timeslot'] = seedData[['hospital', 'date', 'meal']].apply(getDate, axis=1)
seedData = seedData.fillna(value=False)

In [5]:
# Build the restaurant struct
restStruct = {}

def dateCompare(t0, t1='2020-05-18 00:00:00-04'):
    v1 = datetime.strptime(t0, '%Y-%m-%d %H:%M:%S-04')
    v2 = datetime.strptime(t1, '%Y-%m-%d %H:%M:%S-04')
    return v1 > v2

def buildStruct(r):
    restName = r['restName']
    restStruct[restName] = restStruct.get(restName, {"mealIds": {}, "meals": [], "timeslots": [], "menus": []})
    # Create the timeslot and add to list
    timeslotId = str(("%s-%s-%s" % (restName, r['timeslot'], r['hospital'])).__hash__())
    timeslot = {
        "id": timeslotId,
        "hospitalId": hospDict[r['hospital']]['id'],
        "restaurantId": restDict[r['restName']]['id'],
        "date": r['timeslot'],
        "userId": restDict[r['restName']]['userId']
    }
    if dateCompare(r['timeslot']):
        restStruct[restName]['timeslots'].append(timeslot) 
s = seedData.apply(buildStruct, axis=1)

In [6]:
dataOut = json.dumps(restStruct, indent=2)
with open("update_data.json", "w") as outfile: 
    outfile.write(dataOut) 

In [7]:
# Output the csvs
with open("timeslot_data.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['id', 'restaurantId', 'hospitalId', 'userId', 'date'])
    for restName in restStruct:
        for ts in restStruct[restName]['timeslots']:
            writer.writerow([ts['id'], ts['restaurantId'], ts['hospitalId'], ts['userId'], ts['date']])

In [8]:
# Output the csvs
# with open("menu_data.csv", "w") as outfile:
#     writer = csv.writer(outfile)
#     writer.writerow(['timeslotId', 'mealinfoId', 'userId', 'mealName', 'mealDescription', 'allergens', 'dietaryRestrictions'])
#     for restName in restStruct:
#         for ts in restStruct[restName]['menus']:
#             writer.writerow([ts['timeslotId'], ts['mealinfoId'], ts['userId'], ts['mealName'], ts['mealDescription'], ts['allergens'], ts['dietaryRestrictions']])
            

In [9]:
# Output the csvs
# with open("meal_data.csv", "w") as outfile:
#     writer = csv.writer(outfile)
#     writer.writerow(['restaurantId', 'mealinfoId', 'userId', 'mealName', 'mealDescription', 'allergens', 'dietaryRestrictions'])
#     for restName in restStruct:
#         for ts in restStruct[restName]['meals']:
#             writer.writerow([ts['restaurantId'], ts['mealinfoId'], ts['userId'], ts['name'], ts['description'], ts['allergens'], ts['dietaryRestrictions']])
       


In [10]:
    # Generate the meals/menus
#     for i in range(1, 5):
#         mealName = r["mealName%d" % i]
#         if(mealName):
#             mealName = mealName.rstrip()
#             allergens, dietaryRestrictions = [], []
#             if(r['mealAllergens%d' %i]):
#                 allergens = [v.lstrip().rstrip() for v in r['mealAllergens%d' %i].split(',')]
#                 allergens = [allergenLookup[a] for a in allergens]
#             if(r['mealDiet%d' % i]):
#                 dietaryRestrictions = [v.lstrip().rstrip() for v in r['mealDiet%d' % i].split(',')]
#                 dietaryRestrictions = [dietLookup[d] for d in dietaryRestrictions]
#         # Check if meal exists else add
#             if(not mealName in mealDict):
#                 print(mealName, restName);
#                 meal = {
#                     "name": mealName,
#                     "description": r['mealDesc%d' %i],
#                     "allergens" : allergens,
#                     "dietaryRestrictions": dietaryRestrictions,
#                     "mealinfoId": r['meal'].lower() + 'New',
#                     "restaurantId": restDict[r['restName']]['id'],
#                     "userId": restDict[r['restName']]['userId']
#                 }
#                 mealDict[mealName] = meal
#                 restStruct[restName]['meals'].append(meal)
#             # Add the menu to the list
#             menu = {
#                 "timeslotId": timeslotId,
#                 "mealinfoId": r['meal'].lower() + 'New',
#                 "mealName": mealName,
#                 "mealDescription": r['mealDesc%d' %i],
#                 "allergens" : allergens,
#                 "dietaryRestrictions": dietaryRestrictions,
#                 "userId": restDict[r['restName']]['userId']
#             }
#             if dateCompare(r['timeslot']):
#                 restStruct[restName]['menus'].append(menu)

In [11]:
# dietLookup = {
#   "V" : "Vegan",
#   "VG" : "Vegetarian",
#   "GF" : "Gluten Free",
#   "DF" : "Dairy Free",
#   "Vegan": "Vegan",
#   "Vegetarian" : "Vegetarian",
#   "Dairy-Free": "Dairy Free",
#   "Gluten-Free" : "Gluten Free"
# }
# allergenLookup = {
#   'Milk/Dairy' : 'milk/dairy',
#   'Milk/dairy' : "milk/dairy",
#   'Eggs' : "eggs",
#   'Eggs in aioli' : 'eggs',
#   'Pumpkin seeds (seed allergy)' : "pumpkin seeds",
#   'Sesame' : 'sesame',
#   'Sesame Oil' : 'sesame',
#   'Soy' : 'soy',
#   'Dairy' : 'milk/dairy',
#   'Tree nuts' : 'tree nuts',
#   'Fish': 'fish',
#   'Shellfish' : 'shellfish',
#   'Wheat' : 'wheat',
#   'Peanuts': 'peanuts'
# }